### Установка нужных библиотек

In [ ]:
!pip install -q seqeval datasets evaluate ipymarkup transformers faiss-cpu
!pip install -q accelerate -U
!pip install -q transformers[torch]
!git clone https://github.com/AlexKly/Detailed-NER-Dataset-RU.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import os, ast, torch, pathlib, datasets, ipymarkup, transformers, evaluate

### Определение конфигов и параметров

Установим путь к датасету и присвоим метки сущностям NER

In [ ]:
_ROOT = pathlib.Path().resolve()
DIR_DETAILED_NER_DATASET_RU = _ROOT/'Detailed-NER-Dataset-RU'
DIR_DATASET = DIR_DETAILED_NER_DATASET_RU/'dataset'
DIR_OUTPUT = _ROOT/'output'
LABELS = [
    'COUNTRY', 'REGION', 'DISTRICT', 'CITY', 'STREET', 'HOUSE',
    'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME'
]
PARTS = ['B', 'I', 'L', 'U']
LABELS_LIST = [f'{p}-{l}' for l in LABELS for p in PARTS] + ['O']
DOMAINS_MAP = {k: v for v, k in enumerate(LABELS_LIST)}
REVERSE_DOMAINS_MAP = dict((v, k) for k, v in DOMAINS_MAP.items())

Тренировочные данные будут занимать 80% процентов всех данных, тестовые и валидационные - по 10%

In [ ]:
train_ratio = 0.8
test_ratio = 0.1

Загрузим модель и токенизатор специально для задачи NER с HuggingFace

In [ ]:
modelname = 'Babelscape/wikineural-multilingual-ner'
batch_size = 16
tokenizer = transformers.AutoTokenizer.from_pretrained(modelname)

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Определим модель, тренировочные аргументы и функцию-коллатор для данных

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = transformers.AutoModelForTokenClassification.from_pretrained(
    modelname,
    num_labels=len(LABELS_LIST),
    id2label=REVERSE_DOMAINS_MAP,
    label2id=DOMAINS_MAP,
    ignore_mismatched_sizes=True
).to(device)

train_args = transformers.TrainingArguments(
    f'{modelname}-finetuned-ner',
    evaluation_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=False,
    report_to='none'
)

data_collator = transformers.DataCollatorForTokenClassification(tokenizer)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Babelscape/wikineural-multilingual-ner and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([37]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768]) in the checkpoint and torch.Size([37, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Загрузим необходимые метрики

In [ ]:
metric = evaluate.load('seqeval')

Напишем функцию для подсчёта следующих метрик для предсказаний:
- accuracy
- precision
- recall
- f1

In [ ]:
def compute_metrics(p: transformers.trainer_utils.EvalPrediction) -> dict:
    """ Calculate metrics in during training and validation.

    :param p: Pair of predictions and labels.
    :return: Calculated metrics.
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Удаление специальных токенов
    true_predictions = [
        [LABELS_LIST[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [LABELS_LIST[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        'precision': results['overall_precision'],
        'recall': results['overall_recall'],
        'f1': results['overall_f1'],
        'accuracy': results['overall_accuracy'],
    }

### Работа с данными

Загрузим датасет

In [ ]:
df = pd.read_pickle(DIR_DATASET/'detailed-ner_dataset-ru.pickle')
df['tokens'] = df['tokens'].astype(str).apply(lambda x: ast.literal_eval(x))
df['ner_tags'] = df['ner_tags'].astype(str).apply(lambda x: ast.literal_eval(x))
df

,tokens,ner_tags
0,"[dnsmasq, , 3753720, , , , , , , 1, , 0, Mar10...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[2022-09-09, 12:37:10]","[O, O]"
2,"[Повар, судовой]","[O, O]"
3,"[профилирование:, SafeData]","[O, O]"
4,"[Кораблестроение,, океанотехника, и, системоте...","[O, O, O, O, O, O, O]"
...,...,...
7527,[CHANGE_PIN],[O]
7528,[Водоотведение],[O]
7529,[DisplayPromotionalOffersForProducts],[O]
7530,[Союзнефтегаз],[O]


Разобъём его не тренировочную, валидационную и тестовую части

In [ ]:
df = df.sample(frac=1)
train_ds = datasets.Dataset.from_pandas(
    df.iloc[:int(train_ratio * df.shape[0])].reset_index().drop('index', axis=1)
)
test_ds = datasets.Dataset.from_pandas(
    df.iloc[int(train_ratio * df.shape[0]):int((train_ratio + test_ratio) * df.shape[0])].reset_index().drop('index', axis=1)
)
val_ds = datasets.Dataset.from_pandas(
    df.iloc[int((train_ratio + test_ratio) * df.shape[0]):].reset_index().drop('index', axis=1)
)

Протокенизируем получившиеся датасеты

In [ ]:
def tokenize_and_align_labels(ds: datasets.arrow_dataset.Dataset) -> transformers.tokenization_utils_base.BatchEncoding:
    """ Preprare datasets before training model.

    :param ds: Input dataset.
    :return: Prepared tokenized samples and labels.
    """
    label_all_tokens = True
    tokenized_inputs = tokenizer(ds['tokens'], truncation=True, padding=True, is_split_into_words=True, max_length=512)
    labels = list()
    for i, label in enumerate(ds[f'ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = list()
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(DOMAINS_MAP[label[word_idx]])
            else:
                label_ids.append(DOMAINS_MAP[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs['labels'] = labels

    return tokenized_inputs

In [ ]:
tokenized_train_ds = train_ds.map(tokenize_and_align_labels, batched=True, batch_size=batch_size)
tokenized_test_ds = test_ds.map(tokenize_and_align_labels, batched=True, batch_size=batch_size)
tokenized_val_ds = val_ds.map(tokenize_and_align_labels, batched=True, batch_size=batch_size)

Map:   0%|          | 0/6025 [00:00<?, ? examples/s]

Map:   0%|          | 0/753 [00:00<?, ? examples/s]

Map:   0%|          | 0/754 [00:00<?, ? examples/s]

### Тренировка и валидация модели

In [ ]:
%%time

trainer = transformers.Trainer(
    model,
    train_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

# Тренировка и валидация
trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.146744,0.727483,0.700000,0.713477,0.960458
2,0.293100,0.106804,0.787529,0.757778,0.772367,0.973223
3,0.087100,0.090290,0.800895,0.795556,0.798216,0.977258
4,0.048300,0.093792,0.826374,0.835556,0.830939,0.977845
5,0.048300,0.095223,0.851685,0.842222,0.846927,0.978872


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-CITY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-COUNTRY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-LAST_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-REGION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-FIRST_NAME seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metric

CPU times: user 26min 18s, sys: 16.5 s, total: 26min 34s
Wall time: 28min 58s


{'eval_loss': 0.09522262215614319,
 'eval_precision': 0.851685393258427,
 'eval_recall': 0.8422222222222222,
 'eval_f1': 0.846927374301676,
 'eval_accuracy': 0.9788716895312156,
 'eval_runtime': 3.4293,
 'eval_samples_per_second': 219.582,
 'eval_steps_per_second': 13.997,
 'epoch': 5.0}

## Save pretrained model

In [ ]:
trainer.save_model(output_dir=str(DIR_OUTPUT/'ner_ru_model'))

## Загрузка натренированной модели и токенизатора к нему

In [ ]:
my_tokenizer = transformers.AutoTokenizer.from_pretrained(DIR_OUTPUT/'ner_ru_model')
my_model = transformers.AutoModelForTokenClassification.from_pretrained(DIR_OUTPUT/'ner_ru_model')
nlp = transformers.pipeline(task='ner', model=my_model, tokenizer=my_tokenizer, aggregation_strategy='average')

Device set to use cuda:0


In [ ]:
def predict(text: str, confidience: float = 0.5) -> None:
    spans = nlp(text)
    spans_list = [(span['start'], span['end'], span['entity_group']) for span in spans
                  if span['entity_group'] != 'LABEL_0' and span['score'] > confidience]
    ipymarkup.show_span_box_markup(text=text, spans=spans_list)

In [ ]:
predict(text='Сидоров Алексей посетил город Москва на прошлой неделе, но он живет по прописке на ул.Победы д.25 в г.Пенза')
predict(text='Петров Алексей Юрьевич проживает по адресу: г.Москва, ул.Пушкина, д.228, но фактически его страна проживания США')
predict(text='США и Польша - это две страны блока НАТО')

### Поиск похожих по смыслу предложений

In [ ]:
from tqdm.auto import tqdm

Напишем функцю для получения эмбеддингов для всех предложений из тренировочного датасета

In [ ]:
def get_embeddings(ds: datasets.arrow_dataset.Dataset, model=my_model, batch_size=batch_size):
    result = torch.empty(0, model.bert.embeddings.word_embeddings.embedding_dim)
    with torch.no_grad():
        for i in tqdm(range(0, ds.shape[0], batch_size)):
            outputs = model(
                input_ids=torch.as_tensor(ds['input_ids'][i:(i + batch_size)], device=device),
                token_type_ids=torch.as_tensor(ds['token_type_ids'][i:(i + batch_size)], device=device),
                attention_mask=torch.as_tensor(ds['attention_mask'][i:(i + batch_size)], device=device),
                output_hidden_states=True
            ).hidden_states[-1][:, 0, :].cpu()
            result = torch.vstack((result, outputs))
    return result

In [ ]:
embeddings = get_embeddings(tokenized_train_ds)

  0%|          | 0/377 [00:00<?, ?it/s]

Эта функция получает эмбеддинг для поданного на вход предложения

In [ ]:
def get_embedding(phrase, model=model, tokenizer=tokenizer):
    with torch.no_grad():
        return model(
            **{
                key: torch.as_tensor([value], device=device) for key, value in tokenizer(phrase).items()
            }, output_hidden_states=True
        ).hidden_states[-1][:, 0, :].cpu()

Итак, найдём для следующей фразы похожие предложения

In [ ]:
phrase = 'На пост министра обороны была назначена женщина'
embedding = get_embedding(phrase)

In [ ]:
import faiss
k = 10
dimension = embeddings.shape[1]
quantiser = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantiser, dimension, k)

index.train(embeddings)
index.add(embeddings)
index.nprobe = 3

num_clusters = 10  # Количество ближайших соседей
distances, indices = tuple(map(lambda x: x[0], index.search(embedding, num_clusters)))

# Вывод результатов
print("Похожие предложения:")
for i in range(k):
    print(f"{i + 1}. {' '.join(train_ds[int(indices[i])]['tokens'])} (расстояние: {distances[i]})")

Похожие предложения:
1. В отставку были отправлены несколько заместителей министра МВД . (расстояние: 5.488579750061035)
2. Примечательно , что премьер не стал снимать со своих постов двух членов правительства , которые в ноябре 2010г . подверглись жесткой критике верхней палаты парламента , находящейся под контролем оппозиционных сил . (расстояние: 7.880683898925781)
3. Через полчаса после награждения его медаль украли (расстояние: 8.454994201660156)
4. Бывшего уже премьер - министра обвиняли в том , что он перевел на личные счета за границей миллионы долларов , принадлежавшие государству . (расстояние: 8.511103630065918)
5. Дата начала работы сотрудника в компании (расстояние: 8.552425384521484)
6. Эти данные показывают что на журналиста напали в посольстве была борьба а затем его убили (расстояние: 8.779097557067871)
7. Госсекретарь сообщила , что решение о том , кто будет ее преемником , должен принять сам президент . (расстояние: 8.817217826843262)
8. Официально в прокуратуре не с